# Import the necessary libraries, modules, Datasets

## Import Libraries and mModules

In [4]:
import pandas as pd
import transformers
from sklearn.model_selection import train_test_split

## Import Dataset


In [3]:
df_labeled = pd.read_parquet("hf://datasets/qiaojin/PubMedQA/pqa_labeled/train-00000-of-00001.parquet")
df_artificial = pd.read_parquet("hf://datasets/qiaojin/PubMedQA/pqa_artificial/train-00000-of-00001.parquet")
df_unlabeled = pd.read_parquet("hf://datasets/qiaojin/PubMedQA/pqa_unlabeled/train-00000-of-00001.parquet")

#Add final_answer column to the unlabeled dataset
df_unlabeled["final_decision"]=None

df=pd.concat([df_labeled, df_artificial, df_unlabeled], ignore_index=True)

print(f"Labled Shape: {df_labeled.shape}")
print(f"Artificial Shape: {df_artificial.shape}")
print(f"Unlabled: {df_unlabeled.shape}")
print(f"Combined Shape: {df.shape}")

print(df.info())

Labled Shape: (1000, 5)
Artificial Shape: (211269, 5)
Unlabled: (61249, 5)
Combined Shape: (273518, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273518 entries, 0 to 273517
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   pubid           273518 non-null  int32 
 1   question        273518 non-null  object
 2   context         273518 non-null  object
 3   long_answer     273518 non-null  object
 4   final_decision  212269 non-null  object
dtypes: int32(1), object(4)
memory usage: 9.4+ MB
None


## Split Dataset into test and train

### First split the dataframe into train and test

In [5]:
train_set, test_set=train_test_split(df, test_size=0.2, random_state=42)

In [7]:
print(f"Training Set Rows: {train_set.shape}")
print(f"Test Set Rows: {test_set.shape}")

Training Set Rows: (218814, 5)
Test Set Rows: (54704, 5)


### Flatten the dataset
* Currently the column in the datset looks like this(a dictionary with keys-contexts, lables, meshes:
    {
    'contexts': [
        "Use of aspirin is common.",          # Sentence 1
        "We studied 500 patients.",           # Sentence 2
        "Results showed reduced pain."        # Sentence 3
    ],
    'labels': ["BACKGROUND", "METHODS", "RESULTS"],
    'meshes': ["Aspirin", "Pain"]

 * We will check if each row contains a key called contexts and if it does we will will join every sentence from the contexts into a single space separated text

In [9]:
def flatten_context(row):
    if isinstance(row['context'], dict) and 'contexts' in row['context']:
        return " ".join(row['context']['contexts'])

    return ""
    

* We will apply the flatten_context function to every row in the train set and test set
* We apply it to test set also just for uniformity sake.
* We also create/use the .copy() function to break the test_set and train_set links from the original datafram(df)

In [10]:
train_set=train_set.copy()
train_set['full_text']=train_set.apply(flatten_context, axis=1)

test_set=test_set.copy()
test_set['full_text']=test_set.apply(flatten_context, axis=1)

In [16]:
print(train_set.info())
print(f"No of unique pubids: {train_set['pubid'].nunique()}")

<class 'pandas.core.frame.DataFrame'>
Index: 218814 entries, 141060 to 121958
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   pubid           218814 non-null  int32 
 1   question        218814 non-null  object
 2   context         218814 non-null  object
 3   long_answer     218814 non-null  object
 4   final_decision  169916 non-null  object
 5   full_text       218814 non-null  object
dtypes: int32(1), object(5)
memory usage: 10.9+ MB
None
No of unique pubids: 218814


In [15]:
train_set.head()

,pubid,question,context,long_answer,final_decision,full_text
141060,23356518,Do magnolia polyphenols attenuate oxidative an...,{'contexts': ['The bark of magnolia has been u...,This study highlights the important role of NA...,yes,The bark of magnolia has been used in Oriental...
101826,25476117,Are olfactory identification deficits at ident...,{'contexts': ['We have previously reported tha...,These results suggest that impaired OI is not ...,yes,We have previously reported that olfactory ide...
39223,18289138,Are no differences seen in the regional cerebr...,{'contexts': ['Anorexia nervosa (AN) is subdiv...,Abnormalities of the neuronal circuits contain...,yes,Anorexia nervosa (AN) is subdivided into the r...
42170,21725797,Does proton pump inhibitor prophylaxis increas...,{'contexts': ['Stress-related mucosal damage i...,The use of a PPI as a prophylactic treatment a...,yes,Stress-related mucosal damage is an erosive pr...
104449,23731765,Does smooth muscle cell transplantation improv...,{'contexts': ['Damage to smooth muscle has bee...,During the 2-week follow-up period after trans...,yes,Damage to smooth muscle has been the primary c...


# Chunking

In [17]:
import pandas as pd
from transformers import AutoTokenizer

# 1. Setup the Tokenizer (Using a fast standard one, or use your Llama tokenizer)
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

# 2. Define the Chunking Function
def chunk_text_to_df(input_df, text_col='full_text', id_col='pubid', chunk_size=256, overlap=32):
    all_chunks = []
    
    # Calculate the stride (how many tokens to move forward)
    stride = chunk_size - overlap
    
    print(f"Processing {len(input_df)} documents...")
    
    for index, row in input_df.iterrows():
        doc_id = row[id_col]
        text = row[text_col]
        
        # Tokenize the entire text (return tensors='pt' is slower for loops, standard lists are fine here)
        tokens = tokenizer.encode(text, add_special_tokens=False)
        
        # Create sliding window chunks
        chunk_index = 0
        for i in range(0, len(tokens), stride):
            # Define the slice range
            chunk_tokens = tokens[i : i + chunk_size]
            
            # Decode back to text
            chunk_text = tokenizer.decode(chunk_tokens)
            
            # Create the unique chunk ID (e.g., doc_42_chunk_00)
            chunk_id = f"{doc_id}_chunk_{chunk_index:02d}"
            
            # Append to list
            all_chunks.append({
                "chunk_id": chunk_id,
                "document_id": doc_id,
                "chunk_index": chunk_index,
                "chunk_text": chunk_text
            })
            
            chunk_index += 1
            
            # Stop if this was the last chunk (handling exact edge cases)
            if i + chunk_size >= len(tokens):
                break
                
    # Create the final DataFrame
    return pd.DataFrame(all_chunks)

# 3. Apply it to your existing 'train_set'
# Ensure your train_set has 'full_text' and a unique ID column (like 'pubid')
# If you don't have an ID column, create one first: train_set['pubid'] = train_set.index

chunk_df = chunk_text_to_df(
    train_set, 
    text_col='full_text', 
    id_col='pubid',  # Replace with your actual ID column name
    chunk_size=256, 
    overlap=32
)

# 4. View the Result
print(chunk_df.head())
print(f"Total chunks generated: {len(chunk_df)}")

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

Processing 218814 documents...
            chunk_id  document_id  chunk_index  \
0  23356518_chunk_00     23356518            0   
1  23356518_chunk_01     23356518            1   
2  23356518_chunk_02     23356518            2   
3  25476117_chunk_00     25476117            0   
4  25476117_chunk_01     25476117            1   

                                          chunk_text  
0  The bark of magnolia has been used in Oriental...  
1  (DHE) was used to assay superoxide production ...  
2  ted IFNγ±LPS-induced iNOS expression, NO, and ...  
3  We have previously reported that olfactory ide...  
4  up between individuals with a diagnosis of sch...  
Total chunks generated: 421148


## Save the chunked data to disk/working directory as HF Dataset

In [22]:
from datasets import load_from_disk
from datasets import Dataset
hf_chunk_df=Dataset.from_pandas(chunk_df)

# 1. Save the dataset to the output directory
save_path = "./chunked_data_v1"
hf_chunk_df.save_to_disk(save_path)

print(f"✅ Dataset saved successfully to {save_path}")

Saving the dataset (0/1 shards):   0%|          | 0/421148 [00:00<?, ? examples/s]

✅ Dataset saved successfully to ./chunked_data_v1


## Load the saved chunk tabe as a HF Dataset

In [26]:
from datasets import load_from_disk
import os

save_path = "./chunked_data_v1"

# Check if file exists first to avoid errors
if os.path.exists(save_path):
    # Load it directly from disk (takes milliseconds)
    hf_chunked_df = load_from_disk(save_path)
    print("✅ Loaded dataset from disk!")
    
    # Verify it looks right
    print(hf_chunked_df)
else:
    print("⚠️ No saved dataset found. You need to run the chunking code first.")

chunked_df=hf_chunked_df.to_pandas()

print("The dataframe looks like this: ")
print(chunked_df)

✅ Loaded dataset from disk!
Dataset({
    features: ['chunk_id', 'document_id', 'chunk_index', 'chunk_text'],
    num_rows: 421148
})
The dataframe looks like this: 
                 chunk_id  document_id  chunk_index  \
0       23356518_chunk_00     23356518            0   
1       23356518_chunk_01     23356518            1   
2       23356518_chunk_02     23356518            2   
3       25476117_chunk_00     25476117            0   
4       25476117_chunk_01     25476117            1   
...                   ...          ...          ...   
421143  27011950_chunk_00     27011950            0   
421144  20463894_chunk_00     20463894            0   
421145  20463894_chunk_01     20463894            1   
421146  19663554_chunk_00     19663554            0   
421147  19663554_chunk_01     19663554            1   

                                               chunk_text  
0       The bark of magnolia has been used in Oriental...  
1       (DHE) was used to assay superoxide production